In [1]:
import pandas as pd
import numpy as np 
from sklearn import preprocessing
import xgboost as xgb
from sklearn.feature_extraction import DictVectorizer

In [36]:
model = xgb.train({}, D)

TypeError: train() missing 2 required positional arguments: 'params' and 'dtrain'

In [17]:
def wrappers(train_data, test_data):
    
    def xgboost_pred(train,labels,test):
        params = {}
        params["objective"] = "reg:linear"
        params["eta"] = 0.005
        params["min_child_weight"] = 6
        params["subsample"] = 0.7
        params["colsample_bytree"] = 0.7
        params["scale_pos_weight"] = 1
        params["silent"] = 1
        params["max_depth"] = 9


        plst = list(params.items())

        #Using 5000 rows for early stopping. 
        offset = 4000

        num_rounds = 10000
        xgtest = xgb.DMatrix(test)

        #create a train and validation dmatrices 
        xgtrain = xgb.DMatrix(train[offset:,:], label=labels[offset:])
        xgval = xgb.DMatrix(train[:offset,:], label=labels[:offset])

        #train using early stopping and predict
        watchlist = [(xgtrain, 'train'),(xgval, 'val')]
        model = xgb.train(plst, xgtrain, num_rounds, watchlist, early_stopping_rounds=120)
        preds1 = model.predict(xgtest,ntree_limit=model.best_iteration)


        #reverse train and labels and use different 5k for early stopping. 
        # this adds very little to the score but it is an option if you are concerned about using all the data. 
        train = train[::-1,:]
        labels = np.log(labels[::-1])

        xgtrain = xgb.DMatrix(train[offset:,:], label=labels[offset:])
        xgval = xgb.DMatrix(train[:offset,:], label=labels[:offset])

        watchlist = [(xgtrain, 'train'),(xgval, 'val')]
        model = xgb.train(plst, xgtrain, num_rounds, watchlist, early_stopping_rounds=120)
        preds2 = model.predict(xgtest,ntree_limit=model.best_iteration)


        #combine predictions
        #since the metric only cares about relative rank we don't need to average
        preds = (preds1)*1.4 + (preds2)*8.6
        return preds

    #load train and test 
    train  = train_data
    test  = test_data

    labels = train.Hazard
    train.drop('Hazard', axis=1, inplace=True)

    train_s = train
    test_s = test


    train_s.drop('T2_V10', axis=1, inplace=True)
    train_s.drop('T2_V7', axis=1, inplace=True)
    train_s.drop('T1_V13', axis=1, inplace=True)
    train_s.drop('T1_V10', axis=1, inplace=True)

    test_s.drop('T2_V10', axis=1, inplace=True)
    test_s.drop('T2_V7', axis=1, inplace=True)
    test_s.drop('T1_V13', axis=1, inplace=True)
    test_s.drop('T1_V10', axis=1, inplace=True)

    columns = train.columns
    test_ind = test.index


    train_s = np.array(train_s)
    test_s = np.array(test_s)

    # label encode the categorical variables
    for i in range(train_s.shape[1]):
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(train_s[:,i]) + list(test_s[:,i]))
        train_s[:,i] = lbl.transform(train_s[:,i])
        test_s[:,i] = lbl.transform(test_s[:,i])

    train_s = train_s.astype(float)
    test_s = test_s.astype(float)


    preds1 = xgboost_pred(train_s,labels,test_s)

    #model_2 building

    train = train.T.to_dict().values()
    test = test.T.to_dict().values()

    vec = DictVectorizer()
    train = vec.fit_transform(train)
    test = vec.transform(test)

    preds2 = xgboost_pred(train,labels,test)


    preds = 0.47 * (preds1**0.2) + 0.53 * (preds2**0.8)
    
    return preds, preds1, preds2

In [ ]:

from sklearn.cross_validation import KFold
import gini

train = pd.read_csv('../data/train.csv')

scores = []
scores1 = []
scores2 = []
for idx_train, idx_test in KFold(train.shape[0], 5):
    preds, preds1, preds2 = wrappers(train.ix[idx_train], train.ix[idx_test].drop('Hazard', axis=1))
    scores.append(gini.normalized_gini(train.Hazard[idx_test], preds))
    scores1.append(gini.normalized_gini(train.Hazard[idx_test], preds1))
    scores2.append(gini.normalized_gini(train.Hazard[idx_test], preds2))
    
print(np.mean(scores))
print(np.mean(scores1))
print(np.mean(scores2))

Will train until val error hasn't decreased in 120 rounds.
[0]	train-rmse:5.333897	val-rmse:5.603165
[1]	train-rmse:5.320811	val-rmse:5.591259
[2]	train-rmse:5.307675	val-rmse:5.579206
[3]	train-rmse:5.294537	val-rmse:5.566226
[4]	train-rmse:5.281705	val-rmse:5.554331
[5]	train-rmse:5.268994	val-rmse:5.542269
[6]	train-rmse:5.256586	val-rmse:5.530431
[7]	train-rmse:5.244058	val-rmse:5.518928
[8]	train-rmse:5.231432	val-rmse:5.507632
[9]	train-rmse:5.218878	val-rmse:5.495935
[10]	train-rmse:5.206476	val-rmse:5.484262
[11]	train-rmse:5.193877	val-rmse:5.472834
[12]	train-rmse:5.181397	val-rmse:5.461268
[13]	train-rmse:5.169088	val-rmse:5.449983
[14]	train-rmse:5.156974	val-rmse:5.438828
[15]	train-rmse:5.144901	val-rmse:5.427202
[16]	train-rmse:5.132843	val-rmse:5.415715
[17]	train-rmse:5.120923	val-rmse:5.404803
[18]	train-rmse:5.109285	val-rmse:5.394233
[19]	train-rmse:5.097597	val-rmse:5.383402
[20]	train-rmse:5.085852	val-rmse:5.372831
[21]	train-rmse:5.074126	val-rmse:5.362103
[22]	

In [30]:
print(np.mean(scores))
print(np.mean(scores1))
print(np.mean(scores2))

0.161504018312
0.382803988552
0.149775134043


In [33]:
scores1

[0.37908346923232994,
 0.39499705555961601,
 0.38264508634425315,
 0.39085622567853895,
 0.36643810594326298]